<a href="https://colab.research.google.com/github/Rohanrathod7/my-ml-labs/blob/main/14_Model_Validation_in_Python/03_Cross_Validation-learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 3. Cross Validation



In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import datetime as dt
# Import confusion matrix and train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, LogisticRegression, LinearRegression
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier




url = "https://raw.githubusercontent.com/Rohanrathod7/my-ml-labs/main/13_Feature_Engineering_for_Machine_Learning_in_Python/dataset/inaugural_speeches.csv"
# Read the CSV file

# Apply pd.to_numeric only to relevant columns, excluding 'text'
speech_df = pd.read_csv(url)
numeric_cols = ['Name', 'Inaugural Address', 'Date'] # Assuming these should be numeric if possible
for col in numeric_cols:
    speech_df[col] = pd.to_numeric(speech_df[col], errors='coerce')

display(speech_df.head())

,Name,Inaugural Address,Date,text
0,NaN,NaN,NaN,Fellow-Citizens of the Senate and of the House...
1,NaN,NaN,NaN,Fellow Citizens: I AM again called upon by th...
2,NaN,NaN,NaN,"WHEN it was first perceived, in early times, t..."
3,NaN,NaN,NaN,Friends and Fellow-Citizens: CALLED upon to u...
4,NaN,NaN,NaN,"PROCEEDING, fellow-citizens, to that qualifica..."


### The problems with holdout sets

**Two samples**  
After building several classification models based on thetic_tac_toe dataset, you realize that some models do not generalize as well as others. You have created training and testing splits just as you have been taught, so you are curious why your validation process is not working.

After trying a different training, test split, you noticed differing accuracies for your machine learning model. Before getting too frustrated with the varying results, you have decided to see what else could be going on.

In [2]:
url = "https://raw.githubusercontent.com/Rohanrathod7/my-ml-labs/main/14_Model_Validation_in_Python/dataset/tic-tac-toe.csv"
# Read the CSV file

# Apply pd.to_numeric only to relevant columns, excluding non-numeric ones
tic_tac_toe = pd.read_csv(url)


display(tic_tac_toe.head())

,Top-Left,Top-Middle,Top-Right,Middle-Left,Middle-Middle,Middle-Right,Bottom-Left,Bottom-Middle,Bottom-Right,Class
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive


In [3]:
# Create dummy variables using pandas
X = pd.get_dummies(tic_tac_toe.iloc[:,0:9])
y = tic_tac_toe.iloc[:, 9]

# Create training and testing datasets. Use 10% for the test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1111)

# without the holdout set, you cannot truly validate a model. Let's move on to creating two holdout sets.

In [4]:
# Create two different samples of 200 observations
sample1 = tic_tac_toe.sample(200, random_state=1111)
sample2 = tic_tac_toe.sample(200, random_state=1171)

# Print the number of common observations
print(len([index for index in sample1.index if index in sample2.index]))

# Print the number of observations in the Class column for both samples
print(sample1['Class'].value_counts())
print(sample2['Class'].value_counts())

# Notice that there are a varying number of positive observations for both sample test sets.
# Sometimes creating a single test holdout sample is not enough to achieve the high levels of model validation you want.
# You need to use something more robust.

# If our models are not generalizing well or if we have limited data,
# we should be careful using a single training/validation split. You should use the next lesson's topic: cross-validation.

40
Class
positive    134
negative     66
Name: count, dtype: int64
Class
positive    123
negative     77
Name: count, dtype: int64


### Cross-validation

**scikit-learn's KFold()**  
You just finished running a colleagues code that creates a random forest model and calculates an out-of-sample accuracy. You noticed that your colleague's code did not have a random state, and the errors you found were completely different than the errors your colleague reported.

To get a better estimate for how accurate this random forest model will be on new data, you have decided to generate some indices to use for KFold cross-validation.



In [5]:
from sklearn.model_selection import KFold

# Use KFold
kf = KFold(n_splits=5, shuffle=True, random_state=1111)

# Create splits
splits = kf.split(X)


# Print the number of indices
for train_index, val_index in splits:
    print("Number of training indices: %s" % len(train_index))
    print("Number of validation indices: %s" % len(val_index))

# This dataset has 85 rows. You have created five splits - each containing 68 training and 17 validation indices.
# You can use these indices to complete 5-fold cross-validation.

Number of training indices: 766
Number of validation indices: 192
Number of training indices: 766
Number of validation indices: 192
Number of training indices: 766
Number of validation indices: 192
Number of training indices: 767
Number of validation indices: 191
Number of training indices: 767
Number of validation indices: 191


**Using KFold indices**  
You have already created splits, which contains indices for the candy-data dataset to complete 5-fold cross-validation. To get a better estimate for how well a colleague's random forest model will perform on a new data, you want to run this model on the five different training and validation indices you just created.

In this exercise, you will use these indices to check the accuracy of this model using the five different splits. A for loop has been provided to assist with this process.

In [6]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

rfc = RandomForestRegressor(n_estimators=25, random_state=1111)

# Access the training and validation indices of splits
for train_index, val_index in splits:
    # Setup the training and validation data
    X_train, y_train = X[train_index], y[train_index]
    X_val, y_val = X[val_index], y[val_index]

    # Fit the random forest model
    rfc.fit(X_train, y_train)
    predictions = rfc.predict(X_val)
    print("Split accuracy: " + str(mean_squared_error(y_val, predictions)))

#  KFold() is a great method for accessing individual indices when completing cross-validation.
#  One drawback is needing a for loop to work through the indices though. In the next lesson,
#  you will look at an automated method for cross-validation using sklearn

**scikit-learn's methods**   
You have decided to build a regression model to predict the number of new employees your company will successfully hire next month. You open up a new Python script to get started, but you quickly realize that sklearn has a lot of different modules. Let's make sure you understand the names of the modules, the methods, and which module contains which method.

Follow the instructions below to load in all of the necessary methods for completing cross-validation using sklearn. You will use modules:

- metrics
- model_selection
- ensemble

In [7]:
# Instruction 1: Load the cross-validation method
from sklearn.model_selection import cross_val_score

# Instruction 2: Load the random forest regression model
from sklearn.ensemble import RandomForestRegressor

# Instruction 3: Load the mean squared error method
# Instruction 4: Load the function for creating a scorer
from sklearn.metrics import mean_squared_error, make_scorer


**Implement cross_val_score()**  
Your company has created several new candies to sell, but they are not sure if they should release all five of them. To predict the popularity of these new candies, you have been asked to build a regression model using the candy dataset. Remember that the response value is a head-to-head win-percentage against other candies.

Before you begin trying different regression models, you have decided to run cross-validation on a simple random forest model to get a baseline error to compare with any future results.

In [10]:
url = "https://raw.githubusercontent.com/Rohanrathod7/my-ml-labs/main/14_Model_Validation_in_Python/dataset/candy-data.csv"
# Read the CSV file

# Apply pd.to_numeric only to relevant columns, excluding non-numeric ones
candy = pd.read_csv(url)


display(candy.head())


# Define features (X) and target (y)
X = candy.drop(['competitorname', 'winpercent'], axis=1)
y = candy['winpercent']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1111)


,competitorname,chocolate,fruity,caramel,peanutyalmondy,nougat,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent,winpercent
0,100 Grand,1,0,1,0,0,1,0,1,0,0.732,0.860,66.971725
1,3 Musketeers,1,0,0,0,1,0,0,1,0,0.604,0.511,67.602936
2,One dime,0,0,0,0,0,0,0,0,0,0.011,0.116,32.261086
3,One quarter,0,0,0,0,0,0,0,0,0,0.011,0.511,46.116505
4,Air Heads,0,1,0,0,0,0,0,0,0,0.906,0.511,52.341465


In [12]:
rfc = RandomForestRegressor(n_estimators=25, random_state=1111)
mse = make_scorer(mean_squared_error)

# Set up cross_val_score
cv = cross_val_score(estimator=rfc,
                     X=X_train,
                     y=y_train,
                     cv=10,
                     scoring=mse)

# Print the mean error
print(cv.mean())

# You now have a baseline score to build on. If you decide to build additional models or try new techniques,
# you should try to get an error lower than 155.56. Lower errors indicate that your popularity predictions are improving.

135.72463411889825


### Leave-one-out-cross-validation (LOOCV)

 This many observations will definitely slow things down and could be computationally expensive. If you don't have time to wait while your computer runs through 1,000 models, you might want to use 5 or 10-fold cross-validation.

**Leave-one-out-cross-validation**  
Let's assume your favorite candy is not in the candy dataset, and that you are interested in the popularity of this candy. Using 5-fold cross-validation will train on only 80% of the data at a time. The candy dataset only has 85 rows though, and leaving out 20% of the data could hinder our model. However, using leave-one-out-cross-validation allows us to make the most out of our limited dataset and will give you the best estimate for your favorite candy's popularity!

In this exercise, you will use cross_val_score() to perform LOOCV.



In [13]:
from sklearn.metrics import mean_absolute_error, make_scorer

# Create scorer
mae_scorer = make_scorer(mean_absolute_error)

rfr = RandomForestRegressor(n_estimators=15, random_state=1111)

# Implement LOOCV
scores = cross_val_score(rfr, X=X, y=y, cv=y.shape[0], scoring=mae_scorer)

# Print the mean and standard deviation
print("The mean of the errors is: %s." % np.mean(scores))
print("The standard deviation of the errors is: %s." %  np.std(scores))

# You have come along way with model validation techniques.
# The final chapter will wrap up model validation by discussing how to select the best model and give an introduction to parameter tuning.

The mean of the errors is: 9.52044832324183.
The standard deviation of the errors is: 7.349020637882744.
